In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.stats import spearmanr
from sklearn.datasets import fetch_openml
from sklearn.metrics import mean_absolute_error, r2_score,jaccard_score

Step 1: Data Preprocessing

In [2]:
def preprocess_data(df, target_column, categorical_columns=[], n_train=None):
    # Remove rows where target contains NaN values
    df = df.dropna(subset=[target_column])
    
    # Frequency encoding for categorical features
    for col in categorical_columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])

    # Split the dataset into features (X) and target (y)
    X = df.drop(target_column, axis=1)  # Features
    y = df[target_column]  # Target

    # Get number of features (p) and total samples (n_total)
    p = X.shape[1]
    n_total = X.shape[0]

    # Train-test split
    if n_train:
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=n_train, random_state=42)
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Standardize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test, n_total, p, X.columns.tolist()  # Return feature names


Step 2: Enhanced Unravel Algo

In [3]:
class EnhancedUnRAvEL:
    def __init__(self, gp_kernel=None, acquisition_weight=0.5, diversity_lambda=0.3):
        # Define the Gaussian Process Kernel with ARD (Automatic Relevance Determination)
        if gp_kernel is None:
            # Define ARD kernel: C * RBF with automatic relevance determination (ARD)
            gp_kernel = C(1.0, (1e-2, 1e2)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
        
        # Initialize the Gaussian Process Regressor with the defined kernel
        self.surrogate_model = GaussianProcessRegressor(kernel=gp_kernel, normalize_y=True)
        # Acquisition weight for combining different acquisition functions
        self.acquisition_weight = acquisition_weight
        # Lambda for diversity-based penalization
        self.diversity_lambda = diversity_lambda
        
    def inverse_density(self, x, D):
        """
        Correct inverse density calculation.
        Higher density means the point x is close to many points in D.
        """
        if len(D) == 0:
            return 0
        distances = np.linalg.norm(D - x, axis=1) ** 2  # Squared distances
        density = 1 / (1 + np.mean(distances))  # Inverse density formula
        return density

    def adaptive_sampling(self, X_train, X_pool, y_train):
        """
        This method now implements dynamic exploration domain adjustment.
        """
        # Step 1: Calculate uncertainties (exploration) and entropy (exploration)
        uncertainties = self.surrogate_model.predict(X_pool, return_std=True)[1]

        #Step 2: Simulate entropy sampling (exploration encouragement)
        entropy_sampling = np.random.rand(len(X_pool))  # Simulating entropy sampling for exploration

        # Step 3: Define dynamic exploration domain width based on density of samples
        # Density calculation: higher density areas will lead to smaller exploration domains
        #Computing the inverse density for each point in X_pool
        densities = np.array([self.inverse_density(x, X_train) for x in X_pool])

        #Step 4:Calculate dynamic exploration width based on inverse density
        dynamic_width = np.exp(-densities)   # Inverse relationship between density and width

        # Step 4: Compute sampling scores using uncertainties, entropy, and dynamic exploration width
        sampling_scores = uncertainties * (1 - self.acquisition_weight) + entropy_sampling * self.acquisition_weight
        sampling_scores *= dynamic_width  # Adjust by dynamic exploration width

        # Return the sample with the highest score
        return X_pool[np.argmax(sampling_scores)], sampling_scores[np.argmax(sampling_scores)], uncertainties, entropy_sampling, dynamic_width

    def diversity_penalty(self, x, D):
        if len(D) == 0:
            return 0
        return np.mean(np.linalg.norm(D - x, axis=1))

    def acquisition_function(self, x, D, uncertainties, entropy_sampling, dynamic_width, x0, sigma, epsilon, n):
        """
        The hybrid acquisition function that combines:
        - Faithful Uncertainty Reduction (FUR)
        - Diversity Penalization
        - Uses adaptive sampling results directly to avoid recalculating scores
        """
        # Faithful Uncertainty Reduction (FUR) - uncertainty-based score
        # FUR formula as given in the problem
        distance = np.linalg.norm(x - x0)
        sigma_n = np.mean(uncertainties)  # Uncertainty at x based on surrogate model

        # Calculate the FUR score (exploration + uncertainty)
        fur_score = -((distance - sigma * epsilon * np.log(n)) ** 2 / 2) + sigma_n
        # Diversity score is based on how similar `x` is to the existing dataset `D`
        diversity_score = self.diversity_penalty(x, D)
        
        # The acquisition score combines FUR and diversity penalty
        acquisition_score = fur_score + self.diversity_lambda * diversity_score

        # If dynamic width was used to adjust exploration, we adjust acquisition score accordingly
        acquisition_score *= np.mean(dynamic_width)  # Adjust the score based on exploration density

        return acquisition_score

    def train_surrogate(self, X, y):
        self.surrogate_model.fit(X, y)

    def evaluate_stability(self, top_k_features_1, top_k_features_2):
        # Log the top features
        print("Top-k features ARD:", top_k_features_1)
        print("Top-k features LIME:", top_k_features_2)

        # Ensure both arrays have the same size
        if len(top_k_features_1) != len(top_k_features_2):
            min_len = min(len(top_k_features_1), len(top_k_features_2))
            top_k_features_1 = top_k_features_1[:min_len]
            top_k_features_2 = top_k_features_2[:min_len]

        # Calculate Jaccard distance
        intersection = len(set(top_k_features_1) & set(top_k_features_2))
        union = len(set(top_k_features_1) | set(top_k_features_2))
        jaccard_dist = 1 - (intersection / union) if union != 0 else 1

        # Spearman correlation on feature rankings
        spearman_corr, _ = spearmanr(top_k_features_1, top_k_features_2)
        return jaccard_dist, spearman_corr

    def evaluate_fidelity(self, y_true, y_pred):
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        return mae, r2

    def active_learning_routine(self, X_train, y_train, X_test, y_test, max_iterations=10, top_k=5):
        """
        The active learning routine that trains the surrogate model iteratively and performs sampling.
        Incorporates adaptive sampling and acquisition function.
        """
        initial_indices = np.random.choice(len(X_train), size=5, replace=False)
        initial_X = X_train[initial_indices]
        initial_y = y_train.iloc[initial_indices]

        # Train initial surrogate model
        self.train_surrogate(initial_X, initial_y)
        
        dataset_D = initial_X
        y_train = np.delete(y_train, initial_indices)
        X_train = np.delete(X_train, initial_indices, axis=0)

        for iteration in range(max_iterations):
            # Adaptive sampling step, returning all necessary scores
            x_sample, sampling_score, uncertainties, entropy_sampling, dynamic_width = self.adaptive_sampling(X_train, X_test, y_train)

            # Make prediction for the new sample
            y_sample_pred = self.surrogate_model.predict(x_sample.reshape(1, -1))
            if y_sample_pred.ndim == 0:
                y_sample = y_sample_pred
            else:
                y_sample = y_sample_pred[0]

            # Update dataset with new sample and retrain surrogate model
            dataset_D = np.vstack([dataset_D, x_sample])
            initial_y = np.append(initial_y, y_sample)
            self.train_surrogate(dataset_D, initial_y)
            
            # Use acquisition function with adaptive sampling results directly
            acquisition_score = self.acquisition_function(x_sample, dataset_D, uncertainties, entropy_sampling, dynamic_width, initial_X[0], 1.0, np.random.randn(), iteration + 1)
            print(f"Iteration {iteration+1} - Acquisition Score: {acquisition_score}")
            
            # Calculate top-k feature importance using ARD and simulate LIME
            top_k_features_ard = np.argsort(np.abs(self.surrogate_model.kernel_.theta))[-top_k:]
            top_k_features_lime = np.random.choice(X_train.shape[1], top_k, replace=False)  # Simulate LIME results
            
            # Evaluate stability between ARD and LIME
            jaccard_dist, spearman_corr = self.evaluate_stability(top_k_features_ard, top_k_features_lime)
            print(f"Iteration {iteration+1} - Jaccard Distance: {jaccard_dist}, Spearman Correlation: {spearman_corr}")
            
            # Evaluate Fidelity (MAE and R^2)
            y_pred = self.surrogate_model.predict(X_test)
            mae, r2 = self.evaluate_fidelity(y_test, y_pred)
            print(f"Iteration {iteration+1} - MAE: {mae}, R^2: {r2}")

Step 3: Compute the enhanced stability metric

In [4]:
# Function to compute stability metric
def compute_stability_metric(model, X_test, num_samples=10, top_k=10):
    stability_scores_jaccard = []
    stability_scores_spearman = []
    
    # Randomly select num_samples samples
    selected_samples = random.sample(range(len(X_test)), num_samples)
    
    for idx in selected_samples:
        # Simulate top k features selection for each method (replace with actual feature selection logic)
        top_k_features_model_1 = np.random.choice(X_test.shape[1], top_k, replace=False)  # Simulate model 1
        top_k_features_model_2 = np.random.choice(X_test.shape[1], top_k, replace=False)  # Simulate model 2
        
        # Calculate Jaccard distance
        jaccard_dist = 1 - jaccard_score(top_k_features_model_1, top_k_features_model_2, average='macro')
        stability_scores_jaccard.append(jaccard_dist)
        
        # Calculate Spearman correlation
        spearman_corr, _ = spearmanr(top_k_features_model_1, top_k_features_model_2)
        stability_scores_spearman.append(spearman_corr)
    
    # Average both Jaccard distance and Spearman correlation
    avg_jaccard = np.mean(stability_scores_jaccard)
    avg_spearman = np.mean(stability_scores_spearman)
    
    # Combine both metrics into a final stability score (average of both)
    final_stability_metric = (avg_jaccard + avg_spearman) / 2
    
    return final_stability_metric

1)Parkinson's data

In [5]:
# Load Parkinson's dataset
parkinsons_df = pd.read_csv('data/parkinsons.data')
parkinsons_df = parkinsons_df.drop('name', axis=1)
target_column = 'status'
categorical_columns = []
X_train_parkinsons, X_test_parkinsons, y_train_parkinsons, y_test_parkinsons, n_total_parkinsons, p_parkinsons, feature_names_parkinsons = preprocess_data(parkinsons_df, 'status', [], n_train=175)

In [6]:
# Initialize the enhanced algorithm
algorithm1 = EnhancedUnRAvEL()

# Train and evaluate the model on Parkinson's data
algorithm1.active_learning_routine(X_train_parkinsons, y_train_parkinsons, X_test_parkinsons, y_test_parkinsons)


Iteration 1 - Acquisition Score: -16.39266813001795
Top-k features ARD: [0 1]
Top-k features LIME: [ 7  0 10  3  6]
Iteration 1 - Jaccard Distance: 0.6666666666666667, Spearman Correlation: -0.9999999999999999
Iteration 1 - MAE: 0.1, R^2: -0.11111111111111116
Iteration 2 - Acquisition Score: -22.29951408665141
Top-k features ARD: [0 1]
Top-k features LIME: [13  9 19  6 10]
Iteration 2 - Jaccard Distance: 1.0, Spearman Correlation: -0.9999999999999999
Iteration 2 - MAE: 0.1, R^2: -0.11111111111111116
Iteration 3 - Acquisition Score: -13.975839279443642
Top-k features ARD: [0 1]
Top-k features LIME: [ 1 17  0  6 15]
Iteration 3 - Jaccard Distance: 0.6666666666666667, Spearman Correlation: 0.9999999999999999
Iteration 3 - MAE: 0.1, R^2: -0.11111111111111116
Iteration 4 - Acquisition Score: -10.265165674587495
Top-k features ARD: [0 1]
Top-k features LIME: [ 5 15 11  9 16]
Iteration 4 - Jaccard Distance: 1.0, Spearman Correlation: 0.9999999999999999
Iteration 4 - MAE: 0.1, R^2: -0.11111111

c:\Users\ADITI ROY\Desktop\BML_project\.venv\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 0.01. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\ADITI ROY\Desktop\BML_project\.venv\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\ADITI ROY\Desktop\BML_project\.venv\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 0.01. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\ADITI ROY\Desktop

In [7]:
# Run the function for Parkinson's data
stability_metric_parkinsons = compute_stability_metric(algorithm1, X_test_parkinsons)
print(f"Stability Metric for Parkinson's dataset: {stability_metric_parkinsons}")

Stability Metric for Parkinson's dataset: 0.5625541125541126


2)Breast cancer dataset

In [8]:
cancer_df = pd.read_csv('data/cancer.csv')
cancer_df = cancer_df.drop(['id', 'Unnamed: 32'], axis=1)
cancer_df['diagnosis'] = cancer_df['diagnosis'].map({'M': 1, 'B': 0})
target_column = 'diagnosis'
# Updated unpacking for the Cancer dataset
X_train_cancer, X_test_cancer, y_train_cancer, y_test_cancer, n_total_cancer, p_cancer, feature_names_cancer = preprocess_data(cancer_df, 'diagnosis', [], n_train=512)
# Combine preprocessed data into a DataFrame
cancer_train = pd.DataFrame(X_train_cancer)
cancer_train['diagnosis'] = y_train_cancer
cancer_test = pd.DataFrame(X_test_cancer)
cancer_test['diagnosis'] = y_test_cancer

In [9]:
# Initialize the enhanced algorithm
algorithm2 = EnhancedUnRAvEL()

# Train and evaluate the model on Parkinson's data
algorithm2.active_learning_routine(X_train_cancer, y_train_cancer, X_test_cancer, y_test_cancer)


Iteration 1 - Acquisition Score: -44.8671413725407
Top-k features ARD: [0 1]
Top-k features LIME: [28  4 29 15  1]
Iteration 1 - Jaccard Distance: 1.0, Spearman Correlation: -0.9999999999999999
Iteration 1 - MAE: 0.2868452131489382, R^2: 0.2606280680013997
Iteration 2 - Acquisition Score: -8.300562162153156
Top-k features ARD: [0 1]
Top-k features LIME: [15 14  9 28 18]
Iteration 2 - Jaccard Distance: 1.0, Spearman Correlation: -0.9999999999999999
Iteration 2 - MAE: 0.2926868546239992, R^2: 0.23536494417600362
Iteration 3 - Acquisition Score: -22.462878461241218
Top-k features ARD: [0 1]
Top-k features LIME: [18  7 10 27 12]
Iteration 3 - Jaccard Distance: 1.0, Spearman Correlation: -0.9999999999999999
Iteration 3 - MAE: 0.3006273692953365, R^2: 0.21920262733889295
Iteration 4 - Acquisition Score: -6.462464537422692
Top-k features ARD: [0 1]
Top-k features LIME: [ 0  1 27 15 19]
Iteration 4 - Jaccard Distance: 0.0, Spearman Correlation: 0.9999999999999999
Iteration 4 - MAE: 0.299921496

In [10]:
# Run the function for cancer's data
stability_metric_cancer = compute_stability_metric(algorithm2, X_test_cancer)
print(f"Stability Metric for cancer's dataset: {stability_metric_cancer}")

Stability Metric for cancer's dataset: 0.5189605614973263


3) Adult Income Dataset

In [11]:
adult_df = pd.read_csv('data/adult.csv')
adult_df = adult_df.replace('?', np.nan)  # Handle missing values
adult_df = adult_df.dropna()  # Drop any rows with missing values
categorical_columns = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']
target_column = 'income'
adult_df[target_column] = adult_df[target_column].map({'<=50K': 0, '>50K': 1})
X_train_adult, X_test_adult, y_train_adult, y_test_adult, n_total_adult, p_adult, feature_names_adult = preprocess_data(adult_df, target_column, categorical_columns)

In [12]:
# Combine preprocessed data into a DataFrame
adult_train = pd.DataFrame(X_train_adult)
adult_train['income'] = y_train_adult
adult_test = pd.DataFrame(X_test_adult)
adult_test['income'] = y_test_adult

In [13]:
# Initialize the enhanced algorithm
algorithm3 = EnhancedUnRAvEL()

# Train and evaluate the model on adult's data
algorithm3.active_learning_routine(X_train_adult, y_train_adult, X_test_adult, y_test_adult)


Iteration 1 - Acquisition Score: -20.255312444770396
Top-k features ARD: [0 1]
Top-k features LIME: [12  6  8  1  4]
Iteration 1 - Jaccard Distance: 1.0, Spearman Correlation: -0.9999999999999999
Iteration 1 - MAE: 0.34917121992470335, R^2: -0.012480871979881458
Iteration 2 - Acquisition Score: -26.90951297082103
Top-k features ARD: [0 1]
Top-k features LIME: [ 0 11  2 10  5]
Iteration 2 - Jaccard Distance: 0.6666666666666667, Spearman Correlation: 0.9999999999999999
Iteration 2 - MAE: 0.3491734126395933, R^2: -0.012459091910198561
Iteration 3 - Acquisition Score: -106.57704662114062
Top-k features ARD: [0 1]
Top-k features LIME: [1 2 8 4 0]
Iteration 3 - Jaccard Distance: 0.6666666666666667, Spearman Correlation: 0.9999999999999999
Iteration 3 - MAE: 0.3491687506266019, R^2: -0.012508069258232224
Iteration 4 - Acquisition Score: -30.923179800214626
Top-k features ARD: [0 1]
Top-k features LIME: [5 8 2 0 1]
Iteration 4 - Jaccard Distance: 1.0, Spearman Correlation: 0.9999999999999999
I

In [14]:
# Run the function for Adult's data
stability_metric_adult = compute_stability_metric(algorithm3, X_test_adult)
print(f"Stability Metric for Adult's dataset: {stability_metric_adult}")

Stability Metric for Adult's dataset: 0.3828812853812854


4) Boston dataset

In [15]:
boston = fetch_openml(name="boston", version=1, as_frame=True)
X, y = boston.data, boston.target
boston_df = X.copy()
boston_df['MEDV'] = y  # MEDV is the house price (target)
target_column = 'MEDV'
categorical_columns = []
X_train_boston, X_test_boston, y_train_boston, y_test_boston, n_total_boston, p_boston, feature_names_boston = preprocess_data(boston_df, 'MEDV', [], n_train=455)

In [16]:
# Combine preprocessed data into a DataFrame
boston_train = pd.DataFrame(X_train_boston)
boston_train['MEDV'] = y_train_boston
boston_test = pd.DataFrame(X_test_boston)
boston_test['MEDV'] = y_test_boston

In [17]:
# Initialize the enhanced algorithm
algorithm4 = EnhancedUnRAvEL()

# Train and evaluate the model on Boston's data
algorithm4.active_learning_routine(X_train_boston, y_train_boston, X_test_boston, y_test_boston)


Iteration 1 - Acquisition Score: 0.026640230416627435
Top-k features ARD: [0 1]
Top-k features LIME: [10 12  0  6  1]
Iteration 1 - Jaccard Distance: 1.0, Spearman Correlation: 0.9999999999999999
Iteration 1 - MAE: 5.583870436675114, R^2: -0.01679483163580686
Iteration 2 - Acquisition Score: -38.32600212015335
Top-k features ARD: [0 1]
Top-k features LIME: [ 4  1  2  0 11]
Iteration 2 - Jaccard Distance: 0.6666666666666667, Spearman Correlation: -0.9999999999999999
Iteration 2 - MAE: 5.584043860121499, R^2: -0.016821126112669615
Iteration 3 - Acquisition Score: -16.598116015523637
Top-k features ARD: [0 1]
Top-k features LIME: [ 0  5  7 11  3]
Iteration 3 - Jaccard Distance: 0.6666666666666667, Spearman Correlation: 0.9999999999999999
Iteration 3 - MAE: 6.637840542508282, R^2: -0.14351214813795288
Iteration 4 - Acquisition Score: -31.908441676922095
Top-k features ARD: [0 1]
Top-k features LIME: [ 3 10 12  0  6]
Iteration 4 - Jaccard Distance: 1.0, Spearman Correlation: 0.9999999999999

In [18]:
# Run the function for Boston's data
stability_metric_boston = compute_stability_metric(algorithm3, X_test_boston)
print(f"Stability Metric for Boston's dataset: {stability_metric_boston}")

Stability Metric for Boston's dataset: 0.45787296037296044


5) Body fat Dataset

In [19]:
bodyfat_df = pd.read_csv('data/bodyfat.csv')
target_column = 'BodyFat'
X_train_bodyfat, X_test_bodyfat, y_train_bodyfat, y_test_bodyfat, n_total_bodyfat, p_bodyfat, feature_names_bodyfat = preprocess_data(bodyfat_df, 'BodyFat', [], n_train=226)
# Combine preprocessed data into a DataFrame
bodyfat_train = pd.DataFrame(X_train_bodyfat)
bodyfat_train['BodyFat'] = y_train_bodyfat
bodyfat_test = pd.DataFrame(X_test_bodyfat)
bodyfat_test['BodyFat'] = y_test_bodyfat

In [20]:
# Initialize the enhanced algorithm
algorithm5 = EnhancedUnRAvEL()

# Train and evaluate the model on Bodyfat's data
algorithm5.active_learning_routine(X_train_bodyfat, y_train_bodyfat, X_test_bodyfat, y_test_bodyfat)


Iteration 1 - Acquisition Score: -13.583744326419753
Top-k features ARD: [0 1]
Top-k features LIME: [ 2  0  3 12  4]
Iteration 1 - Jaccard Distance: 0.6666666666666667, Spearman Correlation: -0.9999999999999999
Iteration 1 - MAE: 4.861839445890293, R^2: -0.03384491689245017
Iteration 2 - Acquisition Score: -4.1450212446392545
Top-k features ARD: [0 1]
Top-k features LIME: [ 1  6  2  9 10]
Iteration 2 - Jaccard Distance: 0.6666666666666667, Spearman Correlation: 0.9999999999999999
Iteration 2 - MAE: 4.811315039646255, R^2: -0.01329922968709285
Iteration 3 - Acquisition Score: -0.8811939492338686
Top-k features ARD: [0 1]
Top-k features LIME: [13 12  8 10  7]
Iteration 3 - Jaccard Distance: 1.0, Spearman Correlation: -0.9999999999999999
Iteration 3 - MAE: 4.908816689591276, R^2: -0.05365089623650432
Iteration 4 - Acquisition Score: -2.250010555545999
Top-k features ARD: [0 1]
Top-k features LIME: [3 6 5 7 8]
Iteration 4 - Jaccard Distance: 1.0, Spearman Correlation: 0.9999999999999999
It

In [21]:
# Run the function for Bodyfat's data
stability_metric_bodyfat = compute_stability_metric(algorithm3, X_test_bodyfat)
print(f"Stability Metric for Bodyfat's dataset: {stability_metric_bodyfat}")

Stability Metric for Bodyfat's dataset: 0.39640109890109887
